In [1]:
!mkdir train test val
!mkdir train/{images,labels} 
!mkdir test/{images,labels} 
!mkdir val/{images,labels} 

In [2]:
def get_train_unique(ds):
    train_all = sorted( [img for img in os.listdir(ds) if not img.endswith('.json')],key=lambda img: (
    img.split('.rf')[0],
    os.path.getsize(ds+img)
    ))
    last='jfsoifejsf'
    train=[]
    for cur in train_all:
        if last in cur:
            continue
        train.append(cur)
        last=cur.split('.rf')[0]
    return train

# 26 Object detection dataset

In [3]:
import cv2
import os
import json
import pandas as pd

border_size = 20
color = (0, 0, 0)

def create_labels(path_in, path_out,mapping=None):
    # Load JSON annotation file
    json_files = [file for file in os.listdir(path_in) if file.endswith('.json')]

    with open(os.path.join(path_in, json_files[0]), 'r') as f:
        data = json.load(f)
    print(len(json_files))
    # Prepare dataframes from JSON
    df_images = pd.DataFrame(data['images'])
    df_images.rename(columns={'id': 'image_id'}, inplace=True)
    df_categories = pd.DataFrame(data['categories'])
    df_categories.rename(columns={'id': 'category_id'}, inplace=True)
    df_annotations = pd.DataFrame(data['annotations'])
    merged_df = pd.merge(df_annotations, df_categories, how='left', left_on='category_id', right_on='category_id')
    final_df = pd.merge(merged_df, df_images, how='left', left_on='image_id', right_on='image_id')

    # Prepare final dataframe for processing
    df = final_df[['category_id', 'bbox', 'file_name']]
    df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
    df=df.drop('bbox', axis=1)
    df=df.drop_duplicates()
    lables_folder_path='/kaggle/working/'+path_out+'/labels'
    os.makedirs(lables_folder_path,exist_ok =True)
    for file in df['file_name'].unique():
        boxes=df[df['file_name']==file]
        lablel_path='/kaggle/working/'+path_out+'/labels/'+file[:-4]+'.txt'
        with open(lablel_path, 'a') as label_file:
            for id,box in boxes.iterrows():
                class_index = box['category_id']
                x_center = (box['bbox_x'] + box['bbox_width'] / 2 + 10) / 320
                y_center = (box['bbox_y'] + box['bbox_height'] / 2 + 10) / 320
                width = box['bbox_width'] / 320
                height = box['bbox_height'] / 320
                if mapping== None:
                    label_file.write(f"{class_index} {x_center} {y_center} {width} {height}\n")
                elif mapping.get(int(class_index),None)!=None:
                    label_file.write(f"{mapping[int(class_index)]} {x_center} {y_center} {width} {height}\n")

    
    return df

import shutil
def fill_folders(images,folder,ds):
    folder_path='/kaggle/working/'+folder+'/images/'
    os.makedirs(folder_path,exist_ok =True)
    for image in images:
        #os.rename('/kaggle/input/cars-video-object-tracking/images/'+image,'/kaggle/working/'+folder+'/images/'+image)
        shutil.copyfile(ds+image,folder_path+image)
        #if labels:
        #    label=image.split('.')[0]+'.txt'
        #    os.rename('/kaggle/working/labels/'+label,'/kaggle/working/'+folder+'/labels/'+label)


In [4]:
# train = [file for file in os.listdir('/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/train') if not file.endswith('.json')]
train=get_train_unique('/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/train/')
test = [file for file in os.listdir('/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/test') if not file.endswith('.json')]
val = [file for file in os.listdir('/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/valid') if not file.endswith('.json')]

In [5]:
fill_folders(train,'train','/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/train/')
fill_folders(test,'test','/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/test/')
fill_folders(val,'valid','/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/valid/')

In [6]:
df=create_labels('/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/train','train',{10:0,1:1})
df=create_labels('/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/test','test',{10:0,1:1})
df=create_labels('/kaggle/input/26-class-object-detection-dataset/Senior-Design-VIAD-4/valid','valid',{10:0,1:1})


1


/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

1


/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

1


/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']] = pd.DataFrame(df['bbox'].apply(lambda x: pd.Series(x)))
/tmp/ipykernel_35/3142738736.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

# Cars Video Object tracing

In [7]:
from lxml import etree


import os
from os.path import join as opj
from matplotlib import pyplot as plt
import numpy as np
import cv2
import shutil

from tqdm.auto import tqdm

from IPython import display
from PIL import Image

In [8]:
data = {}
w_img=1920
h_img=1080
label2id = {"car": 0, "minivan": 1}
id2label = {v: k for k, v in label2id.items()}
annotations = etree.parse(opj('/kaggle/input/cars-video-object-tracking', "annotations.xml")).getroot()
mapping={0:0,1:1}
for track in annotations.findall('track'):
    label=track.attrib["label"]

    for box in track.findall('box'):
        frame=box.attrib["frame"]
        xtl, ytl = float(box.attrib["xtl"]) / w_img, float(box.attrib["ytl"]) / h_img
        xbr, ybr = float(box.attrib["xbr"]) / w_img, float(box.attrib["ybr"]) / h_img
        xc, yc = (xbr + xtl) / 2, (ybr + ytl) / 2
        yolo_box = [str(xc), str(yc), str(xbr - xtl),str(ybr - ytl)]
        if mapping.get(int(label2id[label]),None)!=None:
            box_data = [str(mapping[int(label2id[label])])] + yolo_box
            if data.get(frame,False):
                data[frame].append(box_data)
            else:
                data[frame]=[box_data]
        

In [9]:
def create_labels(train,test,val):
    processed_data={}
    for id,bboxes in data.items():
        bboxes_str='\n'.join([' '.join(bbox) for bbox in bboxes])
        processed_data[id]=bboxes_str
        name="frame_"+"0"*(6-len(id))+id+'.PNG'
        if name in train:
            folder='train'
        elif name in val:
            folder='val'
        else:
            folder='test'
        with open("/kaggle/working/"+folder+"/labels/frame_"+"0"*(6-len(id))+id+'.txt', "w") as f:
            f.write(bboxes_str)

In [10]:
import random


images=os.listdir('/kaggle/input/cars-video-object-tracking/images')
random.shuffle(images)
boundry=int((len(images)*0.3)/2)
train=images[boundry*2:]
val=images[boundry:boundry*2]
test=images[:boundry]

In [11]:
fill_folders(train,'train','/kaggle/input/cars-video-object-tracking/images/')
fill_folders(test,'test','/kaggle/input/cars-video-object-tracking/images/')
fill_folders(val,'val','/kaggle/input/cars-video-object-tracking/images/')


In [12]:

create_labels(train,test,val)

# Traffic Road Object Detection Polish 12k

In [13]:
yaml_file_path='/kaggle/working/data.yaml'

In [14]:
# train = os.listdir('/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/train/images')
train = [img for img in os.listdir('/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/train/images') if '_aug_' not in img]
test = os.listdir('/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/test/images')
val = os.listdir('/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/valid/images')

In [15]:
fill_folders(train,'train','/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/train/images/')
fill_folders(test,'test','/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/test/images/')
fill_folders(val,'val','/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/valid/images/')

In [34]:
def create_labels(folder,dest,mapping):
    labels=os.listdir(folder)
    for file_name in labels:
        lines=[]
        
        with open(folder+file_name, 'r') as file:
            for line in file:
                info=line.split(' ')
                if not info[0].isnumeric():
                    continue
                if mapping.get(int(info[0]),None) != None:
                    info[0]=str(mapping[int(info[0])])
                    lines.append(' '.join(info))
                
        with open("/kaggle/working/"+dest+"/labels/"+file_name.split('/')[-1], "w") as f:
            f.write('\n'.join(lines))


In [17]:
create_labels('/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/train/labels/','train',{0:0,9:1})
create_labels('/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/test/labels/','test',{0:0,9:1})
create_labels('/kaggle/input/traffic-road-object-detection-polish-12k/road_detection/road_detection/valid/labels/','val',{0:0,9:1})

# Trafic detection project

In [18]:
# train = os.listdir('/kaggle/input/traffic-detection-project/train/images/')
train=get_train_unique('/kaggle/input/traffic-detection-project/train/images/')
test = os.listdir('/kaggle/input/traffic-detection-project/test/images/')
val = os.listdir('/kaggle/input/traffic-detection-project/valid/images/')

In [19]:
fill_folders(train,'train','/kaggle/input/traffic-detection-project/train/images/' )
fill_folders(test,'test','/kaggle/input/traffic-detection-project/test/images/')
fill_folders(val,'val','/kaggle/input/traffic-detection-project/valid/images/')

In [20]:
create_labels('/kaggle/input/traffic-detection-project/train/labels/','train',{2:0,1:1})
create_labels('/kaggle/input/traffic-detection-project/test/labels/','test',{2:0,1:1})
create_labels('/kaggle/input/traffic-detection-project/valid/labels/','val',{2:0,1:1})

# Road Vehicle Images Dataset


In [21]:
train = os.listdir('/kaggle/input/road-vehicle-images-dataset/trafic_data/train/images/')
val = os.listdir('/kaggle/input/road-vehicle-images-dataset/trafic_data/valid/images/')

In [22]:
fill_folders(train,'train','/kaggle/input/road-vehicle-images-dataset/trafic_data/train/images/' )
fill_folders(val,'val','/kaggle/input/road-vehicle-images-dataset/trafic_data/valid/images/')

In [23]:
create_labels('/kaggle/input/road-vehicle-images-dataset/trafic_data/train/labels/','train',{5:0,0:1,4:1,8:1,9:1,10:0,12:1,13:0,16:0,17:0,19:1,20:1})
create_labels('/kaggle/input/road-vehicle-images-dataset/trafic_data/valid/labels/','val',{5:0,0:1,4:1,8:1,9:1,10:0,12:1,13:0,16:0,17:0,19:1,20:1})

# Top-View Vehicle Detection Image Dataset



In [24]:
train = os.listdir('/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/train/images')
val = os.listdir('/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid/images')

In [25]:
fill_folders(train,'train','/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/train/images/' )
fill_folders(val,'val','/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid/images/')

In [26]:
create_labels('/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/train/labels/','train',{0:0})
create_labels('/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/valid/labels/','val',{0:0})

# Cars Detection


In [27]:
train = os.listdir('/kaggle/input/cars-detection/Cars Detection/train/images')
test = os.listdir('/kaggle/input/cars-detection/Cars Detection/test/images')
val = os.listdir('/kaggle/input/cars-detection/Cars Detection/valid/images')

In [28]:
fill_folders(train,'train','/kaggle/input/cars-detection/Cars Detection/train/images/' )
fill_folders(test,'test','/kaggle/input/cars-detection/Cars Detection/test/images/')
fill_folders(val,'val','/kaggle/input/cars-detection/Cars Detection/valid/images/')

In [29]:
create_labels('/kaggle/input/cars-detection/Cars Detection/train/labels/','train',{0:1,1:1,2:0,4:1})
create_labels('/kaggle/input/cars-detection/Cars Detection/test/labels/','test',{0:1,1:1,2:0,4:1})
create_labels('/kaggle/input/cars-detection/Cars Detection/valid/labels/','val',{0:1,1:1,2:0,4:1})

# Vehicle Detection 8 Classes | Object Detection


In [30]:
# train = os.listdir('/kaggle/input/vehicle-detection-8-classes-object-detection/train/images/')
train = get_train_unique('/kaggle/input/vehicle-detection-8-classes-object-detection/train/images/')

In [31]:
fill_folders(train,'train','/kaggle/input/vehicle-detection-8-classes-object-detection/train/images/' )


In [35]:
create_labels('/kaggle/input/vehicle-detection-8-classes-object-detection/train/labels/','train',{2:0,3:1,5:1,6:1})


# Traffic Camera Object Detection


In [36]:
train = os.listdir('/kaggle/input/traffic-camera-object-detection/traffic/train/images')
test = os.listdir('/kaggle/input/traffic-camera-object-detection/traffic/test/images')
val = os.listdir('/kaggle/input/traffic-camera-object-detection/traffic/valid/images')

In [37]:
fill_folders(train,'train','/kaggle/input/traffic-camera-object-detection/traffic/train/images/' )
fill_folders(test,'test','/kaggle/input/traffic-camera-object-detection/traffic/test/images/')
fill_folders(val,'val','/kaggle/input/traffic-camera-object-detection/traffic/valid/images/')

In [38]:
create_labels('/kaggle/input/traffic-camera-object-detection/traffic/train/labels/','train',{0:0})
create_labels('/kaggle/input/traffic-camera-object-detection/traffic/test/labels/','test',{0:0})
create_labels('/kaggle/input/traffic-camera-object-detection/traffic/valid/labels/','val',{0:0})

# Traffic vehicles Object Detection


In [39]:
train = os.listdir('/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/train/')


In [40]:
fill_folders(train,'train','/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/images/train/' )


In [41]:
create_labels('/kaggle/input/traffic-vehicles-object-detection/Traffic Dataset/labels/train/','train',{0:0,5:1,6:1})


In [42]:
yaml_file_path='/kaggle/working/data.yaml'
import yaml
train_data = {
    'train':'/kaggle/working/train',
      'val':'/kaggle/working/val',
      'test':'/kaggle/working/test',
      'names':{
          '0':'Car',
          '1':'Truck/Bus'
      }
     }


print(yaml.dump(train_data))
with open(yaml_file_path, "w") as outfile:
    yaml.dump(train_data, outfile, default_flow_style=False)

names:
  '0': Car
  '1': Truck/Bus
test: /kaggle/working/test
train: /kaggle/working/train
val: /kaggle/working/val



In [43]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [44]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [45]:
print("Train set length:",len(os.listdir('/kaggle/working/train/images')))

Train set length: 28822


In [46]:
model = YOLO("yolo11s.pt") 
results = model.train(data=yaml_file_path, epochs=40, imgsz=640, device=[0, 1])

100%|██████████| 18.4M/18.4M [00:00<00:00, 175MB/s]


Ultralytics 8.3.158 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

100%|██████████| 755k/755k [00:00<00:00, 16.7MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 72.6MB/s]


...

                   all       2751      19008      0.812       0.69      0.779      0.529

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      40/40      3.74G      1.063     0.5994      1.029          6        640: 100%|██████████| 1802/1802 [05:58<00:00,  5.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [00:35<00:00,  4.78it/s]


                   all       2751      19008      0.814       0.69      0.779      0.529

40 epochs completed in 4.475 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 19.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.158 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 172/172 [00:34<00:00,  4.94it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2751      19008      0.811      0.691       0.78      0.529
                   Car       2494      17729      0.844      0.781      0.847      0.562
             Truck/Bus        718       1279      0.779        0.6      0.713      0.496
Speed: 0.2ms preprocess, 3.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train
